In [ ]:
import pandas as pd
import tkinter as tk
import warnings
import os
from tkinter import messagebox
from datetime import datetime
from unidecode import unidecode
import calendar

warnings.filterwarnings('ignore')

rj_path = 'bases/gestiones/RJ'
clasa_path = 'bases/gestiones/CLASA'
mornese_path = 'bases/gestiones/MORNESE'

rj_archivos = [file for file in os.listdir(rj_path) if file.startswith('RJ_REPORTE_CORTES_BBVA_') and file.endswith('.xlsx')]
clasa_archivos = [file for file in os.listdir(clasa_path) if file.startswith('CLASA_REPORTE_CORTES_BBVA_') and file.endswith('.xlsx')]
mornese_archivos = [file for file in os.listdir(mornese_path) if file.startswith('MORNESE_REPORTE_CORTES_BBVA_') and file.endswith('.xlsx')]

In [ ]:
def clean_columns(columns_list: list[str]) -> list[str]:
    return [column.strip().replace('.', '').replace(' ', '_').upper() for column in columns_list]

In [ ]:
def consolidar_archivos(agencia_path: str, archivos: list) -> pd.DataFrame:
    dataframes = []
    for archivo in archivos:
        df = pd.read_excel(os.path.join(agencia_path, archivo), dtype={'CONTRATO': str, 'COD_CENTRAL': str, 'NRO_DOC': str})
        df.columns = clean_columns(df.columns)
        dataframes.append(df)
    
    df_consolidado = pd.concat(dataframes, ignore_index=True)
    df_consolidado.to_excel(os.path.join(agencia_path, f'CONSOLIDADO_GESTIONES_{agencia_path.split('/')[-1]}.xlsx'), index = False)
    return df_consolidado

In [ ]:
import tkinter as tk
from tkinter import messagebox

root = tk.Tk()
root.attributes('-topmost', True)
root.withdraw()

result = messagebox.askquestion('Confirmación', '¿Consolidar archivos de gestiones?', icon='warning')
if result == 'yes':
    archivos = {
        'RJ': (rj_path, rj_archivos),
        'CLASA': (clasa_path, clasa_archivos),
        'MORNESE': (mornese_path, mornese_archivos)
    }
    
    dataframes = {}
    
    for key, (path, file_list) in archivos.items():
        if file_list:
            dataframes[key] = consolidar_archivos(path, file_list)
            print(f'Consolidado {key}:', dataframes[key].shape)
        else:
            print(f'No hay archivos para consolidar en {key}.')

root.destroy()

In [ ]:
root = tk.Tk()
root.attributes('-topmost', True)
root.withdraw()

result = messagebox.askquestion('Confirmación', '¿Leer consolidados?', icon='warning')
if result == 'yes':
    archivos = {
        'RJ': f'{rj_path}/CONSOLIDADO_GESTIONES_RJ.xlsx',
        'CLASA': f'{clasa_path}/CONSOLIDADO_GESTIONES_CLASA.xlsx',
        'MORNESE': f'{mornese_path}/CONSOLIDADO_GESTIONES_MORNESE.xlsx'
    }
    
    dataframes = {}
    
    for key, file_path in archivos.items():
        if os.path.exists(file_path):
            dataframes[key] = pd.read_excel(file_path, dtype={'CONTRATO': str, 'COD_CENTRAL': str, 'NRO_DOC': str})
            print(f'Consolidado {key} leído:', dataframes[key].shape)
        else:
            print(f'El archivo CONSOLIDADO_GESTIONES_{key}.xlsx no existe.')

root.destroy()

In [ ]:
df_rj_test: pd.DataFrame = dataframes['RJ'].copy()
df_clasa_test: pd.DataFrame = dataframes['CLASA'].copy()
df_mornese_test: pd.DataFrame = dataframes['MORNESE'].copy()

df_rj_test['AGENCIA'].replace({'ASESCOM RJ ABOGADOS': 'ASESCOM RJ'}, inplace=True)
df_clasa_test.rename(columns={'TELEFNO': 'TELEFONO'}, inplace=True)
df_clasa_test.rename(columns={'TIPO_DE_MARCACIÓN': 'TIPO_MARCADOR'}, inplace=True)
df_mornese_test.rename(columns={'TIPO_DE_DISCADO': 'TIPO_MARCADOR'}, inplace=True)

In [ ]:
valid_tramos = ['T1', 'T2', 'T3', 'T4']
df_rj_test = df_rj_test[df_rj_test['TRAMO'].isin(valid_tramos)]

In [ ]:
df_rj_test['TIPO_MARCADOR'].value_counts(dropna=False)

In [ ]:
df_clasa_test['TRAMO'].replace({'<1 año': 'T1', 
                                '<2 años': 'T2', 
                                '<3 años': 'T3', 
                                }, inplace=True)

In [ ]:
df_mornese_test['PRODUCTO'].replace({'PRSTAMOS CONSUMO': 'PRESTAMOS CONSUMO', 
                                    'PRSTAMOS PYMES': 'PRESTAMOS PYMES', 
                                    'PRSTAMOS COMERCIALES': 'PRESTAMOS COMERCIALES', 
                                    }, inplace=True)

In [ ]:
df_clasa_test['PRODUCTO'].replace({'PRÉSTAMOS CONSUMO': 'PRESTAMOS CONSUMO', 
                                    'PRÉSTAMOS PYMES': 'PRESTAMOS PYMES', 
                                    'PRÉSTAMOS COMERCIALES': 'PRESTAMOS COMERCIALES', 
                                    'PRÉSTAMO VEHICULAR': 'PRESTAMO VEHICULAR',
                                    }, inplace=True)

In [ ]:
def resumir_consolidado(df: pd.DataFrame, agencia: str) -> pd.DataFrame:
    df['AGENCIA'] = 'ASESCOM RJ' if agencia == 'RJ' else 'CLASA MORA' if agencia == 'CLASA' else 'MORNESE MORA'
    df['HORA_INICIO_LLAMADA'] = pd.to_datetime(df['HORA_INICIO_LLAMADA']).dt.hour.astype('Int64').fillna(0)
    df['ESTADO_LLAMADA'] = df['ESTADO_LLAMADA'].str.upper().str.strip().fillna('NULL')
    df['ESTADO_LLAMADA'] = df['ESTADO_LLAMADA'].apply(unidecode)
    df['MOTIVO_NO_PAGO'] = df['MOTIVO_NO_PAGO'].str.upper().str.strip().fillna('NULL')
    df['MOTIVO_NO_PAGO'] = df['MOTIVO_NO_PAGO'].apply(unidecode)
    df['TIPO_MARCADOR'] = df['TIPO_MARCADOR'].str.upper().replace(' ', '', regex=True).fillna('NULL')
    df['MARCA'] = df['MARCA'].str.upper().replace(' ', '', regex=True)
    df['PDP'] = df.apply(lambda x: 1 if pd.notna(x['FECHA_PDP']) else 0, axis=1)
    df = df[[
        'AGENCIA', 'FECHA_LLAMADA', 'HORA_INICIO_LLAMADA', 'ESTADO_LLAMADA', 'MARCA', 'PDP', 'TRAMO', 'PRODUCTO', 'TIPO_MARCADOR']]
    df.to_excel(f'bases/gestiones/{agencia}/RESUMEN_GESTIONES_{agencia}.xlsx', index=False, sheet_name='BASE')
    return df

In [ ]:
df_rj_final = resumir_consolidado(df_rj_test, 'RJ')
print(df_rj_final.shape)
df_rj_final.head()

In [ ]:
df_clasa_final = resumir_consolidado(df_clasa_test, 'CLASA')
print(df_clasa_final.shape)
df_clasa_final.head()

In [ ]:
df_mornese_final = resumir_consolidado(df_mornese_test, 'MORNESE')
print(df_mornese_final.shape)
df_mornese_final.head()